In [21]:
!pip install pyspark

In [22]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [24]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [25]:
spark

Creating the schema for the CSV file

In [26]:
cschema = StructType([StructField('X', FloatType(), True),
StructField('Y', FloatType(), True),
StructField('RowID', StringType(), True),
StructField('CrimeDateTime', StringType(), True),
StructField('CrimeCode', StringType(), True),
StructField('Location', StringType(), True),
StructField('Description', StringType(), True),
StructField('Inside_Outside', StringType(), True),
StructField('Weapon', StringType(), True),
StructField('Post', StringType(), True),
StructField('District', StringType(), True),
StructField('Neighborhood', StringType(), True),
StructField('Latitude', FloatType(), True),
StructField('Longitude', FloatType(), True),
StructField('GeoLocation', StringType(), True),
StructField('Premise', StringType(), True),
StructField('VRIName', StringType(), True),
StructField('Total_Incidents', IntegerType(), True),
StructField('Shape', StringType(), True)])


###Reading the CSV file

In [27]:
df1 = spark.read.csv('Part1_Crime_data.csv',header=True,sep = ',',schema = cschema)


In [28]:
type(df1)

pyspark.sql.dataframe.DataFrame

####Cache the DataFrame

In [29]:
df1.cache()

DataFrame[X: float, Y: float, RowID: string, CrimeDateTime: string, CrimeCode: string, Location: string, Description: string, Inside_Outside: string, Weapon: string, Post: string, District: string, Neighborhood: string, Latitude: float, Longitude: float, GeoLocation: string, Premise: string, VRIName: string, Total_Incidents: int, Shape: string]

####Count of the rows

In [30]:
df1.count()

513158

####Printing the Schema of the Spark dataframe

In [31]:
df1.printSchema()

root
 |-- X: float (nullable = true)
 |-- Y: float (nullable = true)
 |-- RowID: string (nullable = true)
 |-- CrimeDateTime: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: string (nullable = true)



####Display first 5 rows

In [32]:
df1.show(5)

+---------+---------+-----+--------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+------------+---------------+-----+
|        X|        Y|RowID|       CrimeDateTime|CrimeCode|           Location|   Description|Inside_Outside| Weapon|Post| District|    Neighborhood|Latitude|Longitude|       GeoLocation|        Premise|     VRIName|Total_Incidents|Shape|
+---------+---------+-----+--------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+------------+---------------+-----+
|1420074.1| 594160.6|    1|2022/02/26 04:00:...|       4E|  200 W MONUMENT ST|COMMON ASSAULT|          null|     NA| 124|  CENTRAL|    MOUNT VERNON| 39.2975| -76.6193|(39.2975,-76.6193)|           null|        null|              1| null|
|1411374.2| 589791.4|    2|2022/02/26 01:26:...|

####What are distinct crime codes?

In [33]:
df1.createOrReplaceTempView('Crimecd')

query = """
SELECT distinct Crimecode
FROM Crimecd

"""
print("The distinct crime codes are")
spark.sql(query).show()

The distinct crime codes are
+---------+
|Crimecode|
+---------+
|       3P|
|       3K|
|      3BJ|
|       1A|
|       3M|
|       5F|
|       4B|
|       3B|
|       7A|
|      3NF|
|      3EF|
|       3N|
|       5D|
|       6K|
|      3LO|
|      3AF|
|       7B|
|      3GO|
|     3AJF|
|      8GV|
+---------+
only showing top 20 rows



####Count the number of crimes by the crime codes and order by the resulting counts in descending order

In [34]:
df1.createOrReplaceTempView('crimecount')

query = """
SELECT Crimecode, count(RowID) as count from crimecount group by Crimecode
ORDER BY count DESC

"""
print("The count for each crime code in descending order is : ")
spark.sql(query).show()

The count for each crime code in descending order is : 
+---------+-----+
|Crimecode|count|
+---------+-----+
|       4E|91650|
|       6D|68427|
|       5A|43928|
|       7A|40274|
|       6J|27636|
|       6G|26858|
|       6E|24300|
|       6C|23227|
|       4C|22438|
|       5D|14971|
|      3AF|14739|
|       4B|14460|
|       4A|13226|
|       3B|10737|
|       4D| 7232|
|       5B| 6475|
|       9S| 5443|
|       6F| 5081|
|       5C| 4917|
|       6B| 4106|
+---------+-----+
only showing top 20 rows



####Which neighborhood had most crimes?

In [35]:
df1.createOrReplaceTempView('crimecount')

query = """

SELECT Neighborhood, count(RowID) as Sum_crime_count from crimecount
group by Neighborhood 
order by sum_crime_count desc
limit 1
"""
print("Neighborhood having highest number of crimes : ")
spark.sql(query).show()

Neighborhood having highest number of crimes : 
+------------+---------------+
|Neighborhood|Sum_crime_count|
+------------+---------------+
|    DOWNTOWN|          17799|
+------------+---------------+



####Which month of the year had most crimes?

In [36]:
df1.createOrReplaceTempView('monthcount')

query = """

SELECT substring(CrimeDateTime,6,2) as Month, count(RowID) as Sum_crime_count from monthcount
group by Month 
order by Sum_crime_count desc
limit 1
"""
print("Month of the year with most crimes")
spark.sql(query).show()

Month of the year with most crimes
+-----+---------------+
|Month|Sum_crime_count|
+-----+---------------+
|   08|          46327|
+-----+---------------+



####What weapons were used

In [47]:
df1.createOrReplaceTempView('weapons')

query = """

SELECT DISTINCT Weapon from weapons where Weapon not in ('NA')
"""
print("The weapons used are : ")
spark.sql(query).show()

The weapons used are : 
+-------+
| Weapon|
+-------+
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



####Which weapon was used the most? 

In [39]:
df1.createOrReplaceTempView('Mostusedweapons')

query = """

SELECT Weapon, count(RowID) as count  from Mostusedweapons 
where Weapon Not in ('NA','OTHER') 
group by Weapon 
order by count desc
limit 1
"""
print("The most used weapon was : ")
spark.sql(query).show()

The most used weapon was : 
+-------+-----+
| Weapon|count|
+-------+-----+
|FIREARM|46139|
+-------+-----+

